In [1]:
import pandas as pd
import re
import glob

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('data/projects_complete.csv')

In [5]:
df.shape

(19422, 6)

In [4]:
df.head()

,titulo,FACULTAD,CARRERA,NOMBREPROYEC,counts,URL
0,preparación de nano celulosa y su utilización ...,Facultad de Ciencias Naturales y Matemáticas,Ingeniería Química,Preparación de nano celulosa y su utilización ...,72.0,0
1,monitoreo no intrusivo de cargas eléctricas en...,Facultad de Ciencias Naturales y Matemáticas,Logistica y Transporte,MONITOREO NO INTRUSIVO DE CARGAS ELÉCTRICAS EN...,71.0,0
2,difusión del uso de herramientas tecnológicas ...,"Facultad de Arte, Diseño y Comunicación Audiov...",Diseño Gráfico,Difusión del uso de herramientas tecnológicas ...,62.0,0
3,plan de negocios para la venta de snacks salud...,Facultad de Ciencias Sociales y Humanísticas,Ingeniería en Negocios Internacionales,PLAN DE NEGOCIOS PARA LA VENTA DE SNACKS SALUD...,56.0,0
4,variacion del tiempo de fraguado debido a mixt...,Facultad de Ingeniería en Ciencias de la Tierra,Ingeniería Civil,Variacion del tiempo de fraguado debido a mixt...,56.0,0


In [6]:
df["titulo"].nunique(), df["NOMBREPROYEC"].nunique(), df["FACULTAD"].nunique(), df["CARRERA"].nunique()

(18277, 18672, 10, 78)

In [8]:
df_titulo_count = df["titulo"].value_counts()
df_titulo_count = df_titulo_count[df_titulo_count > 1]
df_titulo_count

titulo
diseño de un equipo para la medición de la biodegradabilidad en películas polimérica y no poliméricas                                                                                                                       6
diseño estudio caracterización y evaluación de la factibilidad de uso de un envase innovador para la industria alimentaria en el mercado ecuatoriano                                                                        6
diseño de un sistema de riego automatizado mediante el suministro de energías no convencionales para la producción de cabras                                                                                                5
determinación de características de las playas de valdivia playa rosada y san pedro para apoyar el manejo de la anidación de tortugas en el país mediante acitividades de turismo responsable                               5
prototipo de control automático de calidad de comida usando imágenes hiperespectrales                    

In [13]:
df[df["titulo"] == df_titulo_count.keys()[0]]

,titulo,FACULTAD,CARRERA,NOMBREPROYEC,counts,URL
544,diseño de un equipo para la medición de la bio...,Facultad de Ingeniería en Mecánica y Ciencias ...,Alimentos,DISEÑO DE UN EQUIPO PARA LA MEDICIÓN DE LA BIO...,24.0,0
574,diseño de un equipo para la medición de la bio...,Facultad de Ingeniería en Mecánica y Ciencias ...,Mecánica,Diseño de un equipo para la medición de la bio...,24.0,0
2141,diseño de un equipo para la medición de la bio...,Facultad de Ciencias Naturales y Matemáticas,Ingeniería Química,DISEÑO DE UN EQUIPO PARA LA MEDICIÓN DE LA BIO...,16.0,0
2658,diseño de un equipo para la medición de la bio...,Facultad de Ingeniería en Electricidad y Compu...,Electrónica y Automatización,DISEÑO DE UN EQUIPO PARA LA MEDICIÓN DE LA BIO...,15.0,0
4264,diseño de un equipo para la medición de la bio...,Facultad de Ciencias de la Vida,Biología,DISEÑO DE UN EQUIPO PARA LA MEDICIÓN DE LA BIO...,11.0,0
6248,diseño de un equipo para la medición de la bio...,Facultad de Ingeniería en Mecánica y Ciencias ...,Ingeniería Mecánica,Diseño de un equipo para la medición de la bio...,2.0,0
